In [1]:
import os, sys
from pathlib import Path
print(os.getcwd())
print(os.getcwd())

parent_dir = Path.cwd().parent.resolve()
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

%load_ext autoreload
%autoreload 2

/home/nevali/cs503_rl/project/dit-ablations
/home/nevali/cs503_rl/project/dit-ablations


In [2]:
import numpy as np
import torch
from PIL import Image


import gc
from accelerate.utils import release_memory

from data.benchmark_data import gather_images
from cache_and_edit.inversion import place_image_in_bounding_box, get_inverted_input_noise, resize_bounding_box, compose_noise_masks



from importlib import reload
import cache_and_edit
reload(cache_and_edit)
from cache_and_edit import *
import cache_and_edit.hooks
reload(cache_and_edit.hooks)


# os.environ['HF_HOME'] = '/dlabscratch1/anmari'
# os.environ['TRANSFORMERS_CACHE'] = '/dlabscratch1/anmari'
# os.environ['HF_DATASETS_CACHE'] = '/dlabscratch1/anmari'


os.environ['HF_HOME'] = '/scratch/nevali'
os.environ['TRANSFORMERS_CACHE'] = '/scratch/nevali'
os.environ['HF_DATASETS_CACHE'] = '/scratch/nevali'

In [3]:
import matplotlib.pyplot as plt

def clear_all_gpu_memory():
    # Run garbage collection
    gc.collect()

    # Get number of available GPUs
    num_gpus = torch.cuda.device_count()
    print(f"Found {num_gpus} GPU(s).")

    # Iterate through each GPU
    for device_id in range(num_gpus):
        with torch.cuda.device(device_id):
            torch.cuda.empty_cache()
            gc.collect()
            torch.cuda.reset_peak_memory_stats()
            torch.cuda.reset_accumulated_memory_stats()
            release_memory()
            torch.cuda.empty_cache()
            gc.collect()
            torch.cuda.ipc_collect()
    print("GPU memory cleared across all available devices.")

def display_side_by_side(image1, image2, image3, titles=None):
    """
    Display three images side by side with optional titles
    
    Args:
        image1, image2, image3: PIL Images to display
        titles: List of titles for each image. If None, no titles are shown.
    """
    
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Display each image
    axes[0].imshow(image1)
    axes[1].imshow(image2)
    axes[2].imshow(image3)
    
    # Set titles if provided
    if titles is not None:
        for i, title in enumerate(titles):
            axes[i].set_title(title)
    
    # Remove axis ticks
    for ax in axes:
        ax.set_xticks([])
        ax.set_yticks([])
    
    plt.tight_layout()
    plt.show()
    return None


clear_all_gpu_memory()

Found 2 GPU(s).
GPU memory cleared across all available devices.


In [4]:
from cache_and_edit.flux_pipeline import EditedFluxPipeline

dtype = torch.float16
pipe = EditedFluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", 
                                    device_map="balanced",
                                    torch_dtype=dtype)
pipe.set_progress_bar_config(disable=True)

# Load cached pipeline
cached_pipe = CachedPipeline(pipe)

Couldn't connect to the Hub: 401 Client Error: Unauthorized for url: https://huggingface.co/api/models/black-forest-labs/FLUX.1-dev (Request ID: Root=1-6829ec52-687fc22d195ab04102b8511d;d727dc06-96a2-4cb6-9a3f-219ca25a0757)

Invalid credentials in Authorization header.
Will try to load from local cache.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [5]:
os_path = "benchmark_images_generations/"
all_images = gather_images(os_path)

Missing images in benchmark_images_generations/Real-Real/0029 a professional photograph of a tomato and strawberries, ultra realistic: fg_mask
Skipping benchmark_images_generations/Real-Real/0029 a professional photograph of a tomato and strawberries, ultra realistic: Not all images are present in benchmark_images_generations/Real-Real/0029 a professional photograph of a tomato and strawberries, ultra realistic
Missing images in benchmark_images_generations/Real-Painting/0021 an oil painting of a tomato, Van Gogh Style: fg_mask
Skipping benchmark_images_generations/Real-Painting/0021 an oil painting of a tomato, Van Gogh Style: Not all images are present in benchmark_images_generations/Real-Painting/0021 an oil painting of a tomato, Van Gogh Style
Loaded 4 images in benchmark_images_generations/


/home/nevali/cs503_rl/project/dit-ablations/data/benchmark_data.py:209: UserWarning: Missing 2 images in benchmark_images_generations/
  warnings.warn(f"Missing {missing_image_counter} images in {image_dir}")


In [6]:
from functools import partial
from cache_and_edit.qkv_cache import TFICONAttnProcessor

NUM_STEPS = 50

# alpha_random = 0.2 nicely working
alpha_random = 0.05


vital_layers = [f"transformer.transformer_blocks.{i}" for i in [0, 1, 17, 18]] + \
                [f"transformer.single_transformer_blocks.{i-19}" for i in [25, 28, 53, 54, 56]]

all_layers = [f"transformer.transformer_blocks.{i}" for i in range(19)] + \
                [f"transformer.single_transformer_blocks.{i-19}" for i in range(19, 57)]


run_on_first = 1

from evaluation.eval import get_scores_for_single_example

methods = ["naive", "tf-icon", "kv-edit", "ours"]



In [ ]:
from cache_and_edit.inversion import compose_noise_masks
from tqdm import tqdm

metrics = {}
for category in tqdm(all_images):
    metrics[category] = []
    for example in all_images[category][:run_on_first]:
        images = []
        labels = []
        for inject_q in [True, False]:
            for inject_k in [True]:
                for inject_v in [True]:
                    print('Composing noise')
                    example_noise = compose_noise_masks(cached_pipe, 
                                example.fg_image, 
                                example.bg_image, 
                                example.target_mask, 
                                example.fg_mask, 
                                option="segmentation1", 
                                num_inversion_steps=50,
                                photoshop_fg_noise=True,)
                    print('Running inject qkv')
                    images = cached_pipe.run_inject_qkv(
                        ["", "", example.prompt],
                        num_inference_steps=NUM_STEPS,
                        seed=42,
                        guidance_scale=3,
                        positions_to_inject=all_layers,
                        positions_to_inject_foreground=vital_layers,
                        empty_clip_embeddings=False,
                        q_mask=example_noise["latent_masks"]["latent_segmentation_mask"],
                        latents=torch.stack(
                                            [
                                            example_noise["noise"]["background_noise"],
                                            example_noise["noise"]["foreground_noise"],
                                            torch.where(
                                                example_noise["latent_masks"]["latent_segmentation_mask"] > 0,
                                                alpha_random * torch.randn_like(example_noise["noise"]["foreground_noise"]) + (1 - alpha_random) * example_noise["noise"]["foreground_noise"],
                                                example_noise["noise"]["background_noise"],
                                            ),
                                            ]
                                        ),
                        processor_class=partial(
                            TFICONAttnProcessor,
                            call_max_times=int(0.4 * NUM_STEPS),
                            inject_q=inject_q,
                            inject_k=inject_k,
                            inject_v=inject_v,
                            ),
                        width=512,
                        height=512,
                        inverted_latents_list = list(zip(example_noise["noise"]["background_noise_list"], example_noise["noise"]["foreground_noise_list"])),
                        tau_b=0.8,
                        bg_consistency_mask=example_noise["latent_masks"]["latent_segmentation_mask"],
                    )
                    print('Computing scores')

                    example.output = images[0][2]
                    scores = get_scores_for_single_example(example, methods)
                    metrics[category].append((example.prompt, inject_q, inject_k, inject_v, scores))
                    images.append(example.output)
                    labels.append(f"Q: {inject_q}, K: {inject_k}, V: {inject_v}" + str(scores))
        print('Plotting')
        # plot all the images for the given example and metrics
        fig, axes = plt.subplots(1, len(images), figsize=(20, 10))
        for i, image in enumerate(images):
            axes[i].imshow(image)
            axes[i].set_title(labels[i])
            axes[i].axis('off')
        plt.tight_layout()
        plt.show()

Photoshopping FG IMAGE


KeyboardInterrupt: 

### NOTE: for each ablation report the results for each of the 4 domains in the benchmark

### Ablation on queries keys and values (remove eachone once at a time), report all metrics

### Ablation on alpha noise (check metric to compute similarity to original image (naive-baseline or only foreground)). How much noise can you add?

### Ablations on the timesteps:
1. fixed tau_alpha and tau_beta --> vary the number of steps for inversion and denoising.
    - hestetic and bg_consistency vs gflops
2. fixed the number of steps and tau_alpha --> vary tau_beta
    - if tau_beta varies we expect a trade-off between hestetic and bg_consist.
3. fixed the number of steps and tau_beta --> vary tau_alpha
    - clip_text_image and dinov2_similarity and hestetic_score (we might expect that the longer you inject the more similarity you get but less hestetic score)


### Classifier free guidance coefficient (check different values now we are using 1.5)

### Effect of the prompt on the output image (for now let's check these two cases):
1. No target prompt
2. Full prompt (folder name)